In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [3]:
cd yolov5

C:\Users\mahen\Intranet\Modele_IA\yolov5


In [4]:
!pip install -r requirements.txt

  Obtaining dependency information for gitpython>=3.1.30 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc457c723931fed3f/GitPython-3.1.43-py3-none-any.whl.metadata
  Obtaining dependency information for pillow>=10.3.0 from https://files.pythonhosted.org/packages/dc/83/1470c220a4ff06cd75fc609068f6605e567ea51df70557555c2ab6516b2c/pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/78/18/7a2e56e2dc45a433dea9e1bf46a65e234294c9c470ccb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


   -------------------- ----------------- 111.4/203.1 MB 281.6 kB/s eta 0:05:26
   -------------------- ----------------- 111.4/203.1 MB 282.3 kB/s eta 0:05:25
   -------------------- ----------------- 111.5/203.1 MB 283.5 kB/s eta 0:05:24
   -------------------- ----------------- 111.5/203.1 MB 283.3 kB/s eta 0:05:24
   -------------------- ----------------- 111.5/203.1 MB 283.8 kB/s eta 0:05:23
   -------------------- ----------------- 111.6/203.1 MB 286.1 kB/s eta 0:05:21
   -------------------- ----------------- 111.6/203.1 MB 286.4 kB/s eta 0:05:20
   -------------------- ----------------- 111.6/203.1 MB 286.5 kB/s eta 0:05:20
   -------------------- ----------------- 111.6/203.1 MB 287.2 kB/s eta 0:05:19
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------

In [1]:
import torch
import cv2
import numpy as np
from facenet_pytorch import InceptionResnetV1
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt

In [2]:
def load_image(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)
    return image

In [3]:
def load_yolo_model():
    # Charger le modèle YOLOv5 small pré-entraîné
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    return model


In [4]:
def load_facenet_model():
    model = InceptionResnetV1(pretrained='vggface2').eval()
    return model

In [5]:
def detect_faces_yolo(model, image):
    # Effectuer la détection des objets
    results = model(image)
    
    # Extraire les boîtes englobantes des objets détectés (en tant que tableau numpy)
    detections = results.xyxy[0].numpy()
    
    return detections

In [6]:
def detect_eyes_opencv(face_region):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return eyes

In [7]:
def align_face(image, face_box, eyes):
    x1, y1, x2, y2 = map(int, face_box)
    face_region = image[y1:y2, x1:x2]
    
    if len(eyes) >= 2:
        # Prendre les deux premiers yeux détectés
        eye_1, eye_2 = eyes[:2]
        eye_1_center = (eye_1[0] + eye_1[2] // 2, eye_1[1] + eye_1[3] // 2)
        eye_2_center = (eye_2[0] + eye_2[2] // 2, eye_2[1] + eye_2[3] // 2)

        # Calcul de l'angle entre les yeux
        if eye_1_center[0] > eye_2_center[0]:
            left_eye, right_eye = eye_2_center, eye_1_center
        else:
            left_eye, right_eye = eye_1_center, eye_2_center
        
        delta_x = right_eye[0] - left_eye[0]
        delta_y = right_eye[1] - left_eye[1]
        angle = np.arctan2(delta_y, delta_x) * 180 / np.pi

        # Aligner le visage en utilisant la rotation
        eyes_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eyes_center, angle, scale=1)
        aligned_face = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
        return aligned_face
    else:
        print("Moins de deux yeux détectés, alignement non effectué.")
        return face_region

In [8]:
def extract_embedding(model, face_region):
    # Redimensionner et préparer l'image pour InceptionResNetV1
    face_img = Image.fromarray(cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB))
    face_img = face_img.resize((160, 160))  # Redimensionner à 160x160 pour FaceNet
    face_tensor = np.array(face_img).astype(np.float32) / 255.0  # Normaliser les pixels
    face_tensor = torch.tensor(face_tensor).permute(2, 0, 1).unsqueeze(0)  # Changer l'ordre des dimensions

    # Obtenir l'embedding
    with torch.no_grad():
        embedding = model(face_tensor)
    return embedding

In [21]:
def draw_rectangle_and_eyes(image, x1, y1, x2, y2, eyes=None):
    # Dessiner un rectangle autour du visage
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Couleur verte (0, 255, 0)
    
    # Si les yeux sont détectés, dessiner des points sur les yeux
    if eyes is not None:
        for (ex, ey, ew, eh) in eyes:
            # Ajuster les coordonnées des yeux pour l'image globale
            adjusted_eye_x = int(ex + x1)
            adjusted_eye_y = int(ey + y1)
            cv2.circle(image, (adjusted_eye_x, adjusted_eye_y), radius=10, color=(0, 0, 255), thickness=-1)  # Couleur rouge pour les yeux

    return image

In [9]:
def display_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [17]:
def process_image_with_facenet_yolo(image_path):
    image = load_image(image_path)
    
    # Charger les modèles
    yolo_model = load_yolo_model()
    facenet_model = load_facenet_model()

    # Détecter les visages
    detections = detect_faces_yolo(yolo_model, image)
    
    if len(detections) == 0:
        print("Aucun visage détecté.")
        return
    
    # Pour chaque visage détecté
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        
        # Filtrer les détections pour garder les personnes uniquement
        if int(cls) == 0 and conf > 0.5:  # Classe 0 correspond à "personne"
            print(f"Visage détecté avec confiance {conf}")
            face_region = image[int(y1):int(y2), int(x1):int(x2)]
            
            # Détecter et aligner le visage
            eyes = detect_eyes_opencv(face_region)
            aligned_face = align_face(image, (x1, y1, x2, y2), eyes)
            
            # Extraire l'embedding avec InceptionResNetV1
            embedding = extract_embedding(facenet_model, aligned_face)
            print(f"Embedding généré: {embedding}")
            
            # Afficher l'image du visage aligné
            display_image(aligned_face)
        else:
            print("Aucun visage valide détecté.")

In [11]:
image = "C:/Users/mahen/Intranet/Modele_IA/images/profil_picture/selena_profile_picture.jpg"

In [12]:
images = "C:/Users/mahen/Intranet/Modele_IA/images/picture/selena_picture_1.jpg"

In [18]:
process_image_with_facenet_yolo(image)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Visage détecté avec confiance 0.9084011316299438
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 6.06704e-02, -3.91767e-02, -4.04874e-02,  1.97538e-02, -6.82385e-02,  1.09077e-02, -1.42266e-02, -3.40124e-02,  1.47941e-03, -4.09394e-02,  9.61531e-02, -3.46455e-02,  3.16642e-02, -7.61875e-03,  1.71091e-02, -4.29378e-02,  3.28028e-02,  2.83367e-02,  9.11454e-02, -4.61295e-02, -1.26163e-02, -6.09578e-02,
          6.47162e-02, -5.27351e-02, -1.12233e-02, -7.99257e-02, -1.83601e-02, -1.46156e-02, -1.20478e-02, -5.16483e-02,  3.63327e-02,  2.01239e-02, -2.80387e-02,  5.72547e-02,  5.03547e-03, -4.21832e-02,  2.59764e-03,  2.86614e-02, -1.46649e-03, -5.82861e-03, -5.45912e-02, -1.88860e-02, -1.10147e-02,  2.71057e-02,
          1.02451e-01,  8.80888e-04,  4.40034e-02, -5.53737e-02,  7.60326e-03, -5.32550e-03,  1.85079e-02,  2.79288e-02,  5.41901e-03,  3.17888e-02, -2.88629e-02,  3.29561e-04,  2.62150e-02, -3.96826e-03, -6.17447e-02,  7.96731e-02, -3.29629e-02

In [19]:
process_image_with_facenet_yolo(images)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Visage détecté avec confiance 0.8587438464164734
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 0.03301, -0.04159, -0.02997,  0.12273, -0.04567,  0.02548, -0.10270,  0.01923,  0.02832, -0.00932,  0.08713,  0.05821, -0.00069,  0.00947, -0.00660, -0.01978,  0.08631,  0.10249, -0.02310, -0.02070, -0.00744,  0.02271,  0.04635, -0.01945, -0.06614, -0.00528, -0.02855,  0.04451, -0.00190,  0.02677, -0.03227,
          0.02169,  0.00385, -0.02527, -0.02883,  0.03533, -0.07852, -0.04811, -0.03020,  0.07730, -0.02151,  0.00925, -0.03631, -0.04097,  0.02510,  0.05692,  0.05348,  0.01610, -0.01651, -0.00699,  0.03487,  0.02418,  0.06555,  0.05335, -0.05882,  0.01480,  0.00215,  0.04705,  0.02113, -0.00204, -0.01252,  0.04489,
         -0.04909,  0.00395,  0.00969,  0.10974,  0.01276, -0.00158,  0.02143, -0.00319,  0.04907,  0.07898, -0.01137,  0.01152,  0.02893, -0.05300, -0.09449,  0.02793, -0.05039,  0.03556,  0.04201,  0.00585,  0.00380,  0.03985,  0.04028,  0

Visage détecté avec confiance 0.7366359233856201
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-0.03725, -0.02770,  0.03468,  0.12963,  0.05349,  0.00344, -0.13053, -0.00147, -0.01936,  0.03574,  0.00120,  0.05857, -0.03714,  0.00724,  0.01840, -0.02593,  0.01897,  0.09565,  0.02543,  0.00510, -0.02093,  0.04275,  0.00293, -0.01467, -0.04413,  0.04435,  0.00611,  0.04663,  0.04756,  0.04917, -0.09167,
         -0.03342, -0.02089, -0.08547, -0.03066, -0.01714, -0.03843, -0.03209, -0.06451,  0.10084, -0.05587,  0.10118,  0.00148, -0.01679, -0.02387,  0.00499,  0.07574,  0.05061, -0.07789,  0.01377,  0.00484,  0.01607,  0.06989,  0.02856, -0.01155,  0.01121, -0.00350,  0.09878,  0.00028, -0.05792,  0.01672,  0.01020,
         -0.03566, -0.05302, -0.02208,  0.10752, -0.03879,  0.00745, -0.03076,  0.00373,  0.05912,  0.07734, -0.00212, -0.01976, -0.01668, -0.01251, -0.02607,  0.02590,  0.01874,  0.00427,  0.08461, -0.00389,  0.04741,  0.04315,  0.01567,  0

Visage détecté avec confiance 0.7028934955596924
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 1.90074e-02, -2.09699e-03, -5.58533e-02,  5.52819e-02, -4.67014e-02,  8.50052e-02, -8.53184e-02,  4.33676e-02, -6.34837e-02,  4.83106e-02,  6.98069e-02,  7.00346e-02,  8.37246e-03, -6.16811e-02, -6.27277e-03, -3.23158e-02,  2.77633e-02,  4.52373e-02,  1.05027e-02, -5.96234e-02, -8.55897e-02,  1.93461e-02,
          5.63279e-02, -5.92171e-02, -2.43651e-02,  8.54182e-03,  2.31948e-02,  1.43147e-02, -4.33271e-02,  3.19700e-02, -1.17833e-02,  3.05185e-02,  3.57952e-02, -2.55767e-03, -4.85384e-02,  1.15371e-02,  2.05922e-02, -1.23215e-02, -1.15594e-02,  8.46118e-03, -7.32713e-03,  6.97667e-03,  2.38634e-02, -6.75324e-03,
          5.07673e-02,  1.39958e-02,  2.16127e-02,  1.01420e-02, -5.17780e-02, -4.19703e-02,  2.66547e-04,  1.18841e-03,  3.17086e-02,  4.60825e-03, -9.07494e-02,  5.28681e-02, -2.41852e-02,  9.65037e-02, -3.82291e-02, -3.54922e-03, -5.16767e-03

Visage détecté avec confiance 0.5949143767356873
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-4.20574e-02, -4.32712e-02, -4.24964e-02,  4.64328e-02,  3.43022e-02, -3.22474e-04,  2.37512e-02,  4.84650e-02,  2.01656e-02, -9.76817e-02, -5.76008e-02,  8.93417e-03,  7.37636e-02,  3.95016e-02,  3.02551e-02, -1.41833e-02, -8.23795e-02,  9.73351e-03, -5.72926e-03, -3.49082e-02, -3.23449e-02, -9.32333e-03,
         -1.19497e-02, -1.02379e-02,  5.60343e-02, -2.87687e-03, -1.07802e-02,  1.60321e-02,  8.01637e-02, -1.59268e-02,  2.11202e-02,  1.05894e-02, -4.07215e-02, -4.08575e-02,  6.88962e-03,  4.81285e-02,  2.22374e-02,  8.90138e-02, -1.24066e-01,  1.45729e-02, -1.16180e-02,  3.83984e-02, -3.31157e-05, -4.65586e-02,
          3.86941e-02, -5.68048e-02,  5.45045e-02, -1.49897e-02, -2.74916e-02, -5.78300e-02, -1.12114e-02, -3.26529e-02,  3.22727e-03, -3.73742e-02, -3.12698e-02,  9.67118e-02, -3.69222e-02,  4.91531e-02,  5.58006e-02, -3.59273e-03,  3.64296e-03

Aucun visage valide détecté.


In [33]:
import torch
print(torch.__version__)

2.5.0+cpu


In [16]:
def display_image(image):
    cv2.imshow("Image avec détection de visages", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()